In [1]:
import sys
import pandas

sys.path.append(".")
import processamento

In [2]:
quantidade_lida = 10000
enem_2019 = pandas.read_csv(
    "./microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv",
    delimiter=";",
    encoding="iso-8859-15",
#    nrows=quantidade_lida,
)
enem_2020 = pandas.read_csv(
    "./microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv",
    delimiter=";",
    encoding="iso-8859-15",
#    nrows=quantidade_lida,
)
enem_2019, enem_2020 = processamento.limpeza(enem_2019, enem_2020)

In [3]:
from pandas_profiling import ProfileReport
def profile(enem,ano):
    #usar apenas a correlação de cramer pq é a unica que funcionar para valores nominais
    profile = enem.profile_report(correlations={
            "pearson": {"calculate": False},
            "spearman": {"calculate": False},
            "kendall": {"calculate": False},
            "phi_k": {"calculate": False},
            "cramers": {"calculate": True},})
    profile.to_file(f"enem-{ano}.html")
    return profile.to_json()

correlacao_2019 = profile(enem_2019,"2019")
correlacao_2020 = profile(enem_2020,"2020")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def gera_tabela_correlacao(profile):
    import json
    d = json.loads(profile)
    faltas = []
    for i in d["correlations"]["cramers"]:
        nomes = i.keys()
        faltas.append(i["falta"])
    correlacao = pandas.DataFrame.from_records(dict(zip(nomes, faltas)),index=[0])
    correlacao = correlacao.sort_values(by=0,axis=1,ascending = False).transpose()
    return correlacao

correlacao_2019 = gera_tabela_correlacao(correlacao_2019)
correlacao_2020 = gera_tabela_correlacao(correlacao_2020)

correlacao = pandas.concat([correlacao_2019,correlacao_2020],keys=["2019","2020"],axis=1)
correlacao.to_csv("correlacao.csv")

In [6]:
def crosstab(variavel_explicativa,variavel_resposta):
    contigencia = pandas.crosstab(variavel_explicativa, variavel_resposta,normalize='all')
    return contigencia
    

def salvar_grafico(contigencia,nome):
    plot = contigencia.plot.bar(title=nome)
    fig = plot.get_figure()
    fig.savefig(f"grafico_{nome}.png")
    del
    
    
for (_,coluna_2019),(_,coluna_2020) in zip(enem_2019.iteritems(),enem_2020.iteritems()):
    contigencia_2019 = crosstab(coluna_2019,enem_2019["falta"])
    contigencia_2020 = crosstab(coluna_2020,enem_2020["falta"])
    contigencia_diferenca = (contigencia_2020-contigencia_2019)[1]
    if(coluna_2020.name in ["CO_MUNICIPIO_ESC","CO_MUNICIPIO_PROVA","CO_MUNICIPIO_PROVA","CO_MUNICIPIO_ESC"]):
        contigencia_diferenca = contigencia_diferenca[~contigencia_diferenca.isnull()]
        contigencia_diferenca = contigencia_diferenca.sort_values()
        contigencia_diferenca = pandas.concat([contigencia_diferenca.head(10),contigencia_diferenca.tail(10)])

    salvar_grafico(pandas.DataFrame(contigencia_diferenca),coluna_2020.name)

/home/dnl/.local/lib/python3.10/site-packages/pandas/plotting/_matplotlib/core.py:345: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)
